# Extracción de datos
En este notebook se tratará la extracción de datos de sitios web como Reddit o Github, mediante APIs expresamente hechas por el sitio, o funciones de Python que servirán para realizar las peticiones.

Para empezar, se cargarán los ajustes definidos en el apartado anterior junto con la importación de algunas librerías básicas necesarias para comenzar. 

In [1]:
import sys, os

#Reset del entorno virtual al iniciar la ejecución
%reset -f

#Carga del archivo setup.py
%run -i setup.py

#Carga del archivo settings.py
#%run "$BASE_DIR/settings.py"
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

You are working on a local system.
Files will be searched relative to "..".


## Extracción de datos con la librería *request*

Para este primer ejemplo se va a hacer uso de la librería *request* que se incluye con la instalación de python. Esta librería es el método más básico para acceder y extraer información a través de una API, pero al mismo tiempo es una herramienta muy potente.

Véamos a continuación como se realiza una petición GET en la que se tratará de listar todos los repositorios que cuenten con unas características  determinadas. En este caso, por ejemplo, vamos a listar todos los repositorios relacionados con el protocolo Zigbee que, al no usar el token de autenticación de un usuario registrado, nos devolverá aquellos repositorios que sean públicos, si se quieren listar también aquellos privados, hay que proporcionar el token de acceso.

In [2]:
import requests
import json

response = requests.get('https://api.github.com/search/repositories',
    params={'q': 'zigbee'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'})

print(response.status_code)
#print(response.json())

200


Para poder entender de forma más clara la lista de repositorios obtenida, vamos a dar formato Markdown al archivo json y se mostrarán los 5 primeros resultados de la respuesta obtenida:

In [3]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

for item in response.json()['items'][:5]:
    printmd('**' + item['name'] + '**' + ': repository ' +
            item['text_matches'][0]['property'] + ' - \"*' +
            item['text_matches'][0]['fragment'] + '*\" matched with ' + '**' +
            item['text_matches'][0]['matches'][0]['text'] + '**')

**zigbee2mqtt.io**: repository name - "*zigbee2mqtt.io*" matched with **zigbee2mqtt**

**zigbee2mqtt**: repository description - "*Zigbee 🐝 to MQTT bridge 🌉, get rid of your proprietary Zigbee bridges 🔨*" matched with **Zigbee**

**zigbee**: repository name - "*zigbee*" matched with **zigbee**

**zigbee-herdsman-converters**: repository description - "*Collection of device converters to be used with zigbee-herdsman*" matched with **zigbee**

**hassio-zigbee2mqtt**: repository name - "*hassio-zigbee2mqtt*" matched with **zigbee2mqtt**

### Listar comentarios del apartado *Issues* de un repositorio
Es posible listar todos los comentarios del apartado Issues de un repositorio si así se especifica en la petición el nombre del repositorio y el dueño del mismo:

In [4]:
response = requests.get(
    'https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments')
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 403
Number of comments 2


Se observa que únicamente ha recopilado 30 comentarios, esto es porque la API de github limita el número de elementos para cada respuesta. Si mostramos por pantalla los enlaces de la respuesta obtendremos el número de páginas contenidas en la respuesta.

In [5]:
response.links

{}

### Paginación
Se usa la paginación para limitar el número de elementos en una que se devolverán tras una petición.

Al mostrar los links de *response* se ve que proporciona el enlace a la siguiente página a la respuesta y a la última del total.

Para obtener todos los resultados se debe definir una función que llame a la siguiente página, y así hasta que se hayan procesado todas de forma recursiva.

Del mismo modo, importaremos *Pandas* a nuestro código para transformar los datos obtenidos en un Data Frame.

En el siguiente ejemplo se muestra lo anteriormente explicado además de mostras por pantalla el número de filas que se desee del Data Frame:

In [6]:
import pandas as pd

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json


out = get_all_pages(
    "https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments",
    params={
        #Como parámetros indicamos la fecha desde la cuál queremos obtener los resultados,
        #el orden (de creación) y la dirección
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept': 'application/vnd.github.v3+json'})

#Los resultados obtenidos los transformamos en un Data Frame para su posterior análisis
df = pd.DataFrame(out)
pd.set_option('display.max_colwidth', 1)

# #Muestra por pantalla el total de resultados
print(df['body'].count())

# #Muestra por pantalla algún ejemplo de los resultados obtenidos
df[['id', 'created_at', 'body']].sample(10, random_state=42)

KeyError: 'body'

**La razón por la que aparece este error en la ejecución es porque se ha superado el límite de peticiones y la respuesta es vacía. Esto se solventa en los siguientes apartados, en los que se usa un token de autenticación para un mayor límite y poder realizar diversas ejecuciones asegurando obtener una respuesta**

### Límites de las APIs
Las APIs tienen un límite a la hora de devolver los resultados, por ejemplo, se pueden haber obtenido 400 comentarios, pero si se accede al sitio del que estos han sido extraídos, cabe la posibilidad de que el total sea ampliamente mayor.

Por esto, se va a definir una función que impida sobrecargar el servidor al que se le realizan las peticiones, disminuyendo la velocidad entre una petición y la siguiente y asegurarnos de que toda la información que se ha solicitado sea descargada correctamente.

In [7]:
from datetime import datetime
import time

def handle_rate_limits(response):
    now = datetime.now()
    reset_time = datetime.fromtimestamp(
        #X_RateLimit indica cuántas peticiones se puden realizar por unidad de tiempo
        int(response.headers['X-RateLimit-Reset']))
    
    #X-RateLimit-Remaining indica cuántas peticiones pueden aún hacerse
    #sin superar el límite establecido
    remaining_requests = response.headers['X-RateLimit-Remaining']
    remaining_time = (reset_time - now).total_seconds()
    intervals = remaining_time / (1.0 + int(remaining_requests))

    print('Esperando por ', intervals)
    time.sleep(intervals)

    return True

La librería *requests* no contempla una función que permita reintentar la petición en caso de error, aún con esto, se puede implementar gracias a la librería *HTTPAdapter*.

In [8]:
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

retry = Retry(
    #5 reintentos
    total=5,
    #Códigos de erros los cuáles si se reciben, se reintentará
    status_forcelist=[500, 503, 504],
    #Retraso entre reintentos después del segundo intento
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("https://", retry_adapter)

response = http.get('https://api.github.com/search/repositories',
                    params={'q': 'zigbee'})

for item in response.json()['items'][:5]:
    print(item['name'])

zigbee2mqtt.io
zigbee2mqtt
zigbee
zigbee-herdsman-converters
hassio-zigbee2mqtt


Juntando las funciones definidas en el apartado de paginación junto con las definidas en este último, quedaría como resultado algo así:

In [ ]:
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

retry = Retry(
    #5 reintentos
    total=5,
    #Códigos de erros los cuáles si se reciben, se reintentará
    status_forcelist=[500, 503, 504],
    #Retraso entre reintentos después del segundo intento
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("https://", retry_adapter)

def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json

#Función que lee el token de autenticación de github de un .txt
def load_token(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            token = file.read().strip()  # .strip() elimina espacios y saltos de línea
            return token
    except FileNotFoundError:
        raise Exception(f"El archivo {filepath} no se encontró. Asegúrate de que existe y contiene el token.")

token = load_token('token.txt')

out = get_all_pages(
    "https://api.github.com/repos/zigbee2mqtt/hassio-zigbee2mqtt/issues/comments",
    params={
        #Como parámetros indicamos la fecha desde la cuál queremos obtener los resultados,
        #el orden (de creación) y la dirección
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    #Introduzco el token de autenticación para que no supere el número de peticiones y puede realizar varias ejecuciones seguidas
    headers={'Authorization': f'token {token}',
            'Accept': 'application/vnd.github.v3+json'}
            )

#Los resultados obtenidos los transformamos en un Data Frame para su posterior análisis
df = pd.DataFrame(out)
#Guardo el dataframe en formato csv/json para su uso en operaciones posteriores
df.to_csv('output.csv', index=False)
df.to_json('output.json', orient='records', lines=True)

pd.set_option('display.max_colwidth', 1)

# #Muestra por pantalla el total de resultados
#print(df['body'].count())

# #Muestra por pantalla algún ejemplo de los resultados obtenidos
df[['id', 'created_at', 'body']].sample(5, random_state=42)

,id,created_at,body
840,1620375319,2023-07-04T14:39:43Z,"@fthiery you will need 2 coordinators to start the second instance, devices not, you can manually edit data/database.db and add entries from your prod setup."
2234,897745813,2021-08-12T15:41:55Z,Can I close this issue? Or is it still relevant?
443,2081183179,2024-04-27T21:09:04Z,I am having this exact log output and fail to start after trying to move my Sonoff Zigbee stick over to a Remote Adapter using ser2net on a raspberry pi. Any luck on this? I can't seem to find anything.
836,1621714839,2023-07-05T13:02:33Z,"> I was able to fix this from the terminal, here's what I did:\r\n> \r\n> ```\r\n> cp -Rp /root/config/zigbee2mqtt /root/config/zigbee2mqtt.bak\r\n> cp -Rp /root/config/zigbee.db /root/config/zigbee.db.bak # probably not needed\r\n> \r\n> # Uninstall add-on from the GUI\r\n> \r\n> # Re-Install add-on from the GUI\r\n> \r\n> # Stop the add-on from the GUI\r\n> \r\n> cp -Rp /root/config/zigbee2mqtt.bak/* /root/config/zigbee2mqtt/\r\n> \r\n> # Start the add-on from the GUI\r\n> ```\r\n> \r\n> My configuration was preserved. I dunno if this is the cleanest solution, but there it is.\r\n\r\nhave to add back manually the configuration settings and it's working again !"
2589,786512624,2021-02-26T09:07:35Z,stupid me used the new config file and never changed log output to info.
